In [1]:
import numpy as np
from scipy import linalg
# Load anisotropic strain file
def read_strain_data(filename):
    data = np.loadtxt(filename, delimiter=',', skiprows=1)
    angles = data[:, 0]
    strain_components = data[:,1:]
    return angles, strain_components

# Strain-related energy terms
def calculate_strain_energy(a_v, b_v, d_v, epsilon):
    epsilon_xx, epsilon_yy, epsilon_zz, epsilon_xy, epsilon_yz, epsilon_xz = epsilon
    P_eps = a_v * (epsilon_xx + epsilon_yy + epsilon_zz)
    Q_eps = b_v * (epsilon_zz - (epsilon_xx + epsilon_yy)/2)
    R_eps = (np.sqrt(3)/2) * b_v * (epsilon_xx - epsilon_yy) - 1j * d_v * epsilon_xy
    S_eps = -d_v * (epsilon_xz - 1j * epsilon_yz)
    print(epsilon_xx, epsilon_yy, epsilon_zz, epsilon_xy, epsilon_yz, epsilon_xz)
    return P_eps, Q_eps, R_eps, S_eps

# 4*4 Luttinger Hamiltonian without spin–orbit coupling from [1] SUN Y, THOMPSON S E, NISHIDA T. Strain Effect in Semiconductors: Theory and Device Applications[M/OL]. Boston, MA: Springer US, 2010[2024-05-27]. https://link.springer.com/10.1007/978-1-4419-0552-9. DOI:10.1007/978-1-4419-0552-9.

def build_kp_matrix(P_eps, Q_eps, R_eps, S_eps, Delta0):
    H = np.zeros((4, 4), dtype=complex)
    
    H[1, 1] = H[2, 2] = -P_eps + Q_eps
    H[0, 0] = H[3, 3] = -P_eps - Q_eps
    
    H[0, 1] = (S_eps)
    H[0, 2] = -R_eps
    H[0, 3] = 0
    H[1, 2] = 0
    H[1, 3] = -R_eps
    H[2, 3] = -(S_eps)

    # Hermitian matrix
    for i in range(4):
        for j in range(i+1, 4):
            H[j, i] = np.conj(H[i, j])

    return H

# Derive the calculation of the energy shift
def write_results(filename, angles, eigenvalues, Delta_Ec, Delta_Ev1, Delta_Ev2, Eg):
    with open(filename, 'w') as f:
        f.write("Angle,E1(eV),E2(eV),E3(eV),E4(eV),Ec,Ehh,Elh,Eg\n")
        for i, angle in enumerate(angles):
            eigenvalues_str = ",".join(f"{ev:.6f}" for ev in eigenvalues[i])
            f.write(f"{angle:.2f},{eigenvalues_str},{Delta_Ec[i]:.6f},{Delta_Ev1[i]:.6f},{Delta_Ev2[i]:.6f}, {Eg[i]:.6f}\n")
    print(f"Results written to {filename}")

In [2]:
# Deformation potentials for GaAs
a_c = -7.17  # eV
a_v = -1.16  # eV
b_v = -2.0  # eV
d_v = -4.8  # eV
Delta0 = 0.341  # eV

# Load angle-dependent anisotropic strain data
try:
    angles, strain_components = read_strain_data('GaAsStrain_fromFEM_Simulation.dat')
    print("Data loaded successfully.")
except Exception as e:
    print(f"Error reading data: {e}")


eigenvalues_list = []
Delta_Ec_list = []
Delta_Ehh_list = []
Delta_Elh_list = []
Delta_Eg_list = []

Data loaded successfully.


In [3]:

for i, epsilon in enumerate(strain_components):
  print(f"Processing strain component {i+1}/{len(strain_components)}")
  P_eps, Q_eps, R_eps, S_eps = calculate_strain_energy(a_v, b_v, d_v, epsilon)
  H = build_kp_matrix(P_eps, Q_eps, R_eps, S_eps, Delta0)

  eigenvalues, eigenvectors = linalg.eigh(H)
  eigenvalues_list.append(eigenvalues.real)

  epsilon_xx, epsilon_yy, epsilon_zz, epsilon_xy, epsilon_yz, epsilon_xz = epsilon

  # Analytical solutions for the electric conduction band change
  Delta_Ec = a_c * (epsilon_xx + epsilon_yy + epsilon_zz)
  Delta_Ec_list.append(Delta_Ec)

  # Analytical solutions for the eigenvalues of heavy hole band change
  Delta_Ehh = - P_eps + np.sqrt(np.abs(Q_eps)**2+np.abs(S_eps)**2+np.abs(R_eps)**2)
  Delta_Ehh_list.append(Delta_Ehh)  
  # Analytical solutions for the eigenvalues of light hole band change
  Delta_Elh = - P_eps - np.sqrt(np.abs(Q_eps)**2+np.abs(S_eps)**2+np.abs(R_eps)**2)
  Delta_Elh_list.append(Delta_Elh)  
  Delta_Eg_list.append(Delta_Ec-Delta_Ehh)

write_results('Strain_induced_EnergyShift.dat', angles, eigenvalues_list, Delta_Ec_list, 
              Delta_Ehh_list, Delta_Elh_list, Delta_Eg_list)

Processing strain component 1/19
-1.31095623007467e-06 1.15687367425555e-05 -4.70282753322535e-06 2.84432539848742e-07 3.15451129451545e-07 7.77784117168305e-08
Processing strain component 2/19
1.10212707257628e-06 1.77094396990724e-05 -8.86928092791684e-06 -1.11802628549311e-06 2.75722746499304e-07 5.51334692093707e-06
Processing strain component 3/19
-4.76819610209451e-06 3.41483206845648e-05 -1.31541492745727e-05 -2.41051640465913e-06 2.12817753508238e-07 1.02449772094699e-05
Processing strain component 4/19
-1.67148181039482e-05 5.79966181497278e-05 -1.75179188347547e-05 -3.1316757878639e-06 8.15109583719055e-08 1.09968567861386e-05
Processing strain component 5/19
-3.07417806626624e-05 8.38790181897971e-05 -2.17430330721871e-05 -3.03136849631418e-06 -1.28300004110621e-07 5.77502511111267e-06
Processing strain component 6/19
-4.23174176146095e-05 0.000105695461540971 -2.55832353126822e-05 -2.10037418123111e-06 -3.75073548937124e-07 -5.64330997386531e-06
Processing strain component 